# EQR Analysis

By 503 Team 4

## Background

## Code

In [ ]:
#import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import altair as alt
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook_connected"

# alt.data_transformers.enable('json')
alt.data_transformers.enable('default',max_rows=None)

In [ ]:
#load data

csvs = []

file_path = '../data/quarterly_financials/'
for file in os.listdir(file_path):
    if file.endswith('.csv'):
        tmp_file = pd.read_csv(os.path.join(file_path, file))
        csvs.append(tmp_file)
df = pd.concat(csvs, ignore_index=True)

In [ ]:
#preprocessing
df['REPDTE'] = df['REPDTE'].astype(str)
#add year and quarter
df['YEAR'] = df['REPDTE'].str[:4]
df['QUARTER'] = ''

quarters_dict = {'0331': 'Q1', '0630': 'Q2', '0930': 'Q3', '1231': 'Q4'}
df['QUARTER'] = df['REPDTE'].str[-4:].map(quarters_dict).fillna('ERROR')

df['YEAR'] = df['YEAR'].astype(str)
df['QUARTER'] = df['QUARTER'].astype(str)
df['QUARTER_YEAR'] = df['YEAR'] + ' ' + df['QUARTER']

## Top and Bottom 20 Histogram

In [ ]:
df['COUNT'] = 1
df_eqr_overall = df.groupby('CERT').agg({'EQR' : 'mean', 'NAME': 'first', 'COUNT': 'sum', 'STNAME': 'first', 'CERT': 'first'})
df_eqr_overall = df_eqr_overall[df_eqr_overall['COUNT'] >= 100]

In [ ]:
mean_value = df_eqr_overall['EQR'].mean()
print(df_eqr_overall['EQR'].describe())

largest = df_eqr_overall.nlargest(20, 'EQR')
smallest = df_eqr_overall.nsmallest(20, 'EQR')
plot_eqr_df = pd.concat([largest, smallest], ignore_index=True)

mean_row = pd.DataFrame({'EQR': [mean_value], 'NAME' : ['MEAN']})
plot_eqr_df = pd.concat([plot_eqr_df, mean_row], ignore_index=True)
plot_eqr_df = plot_eqr_df.sort_values(by='EQR', ascending=True)

In [ ]:
selection = alt.selection_single(fields=['NAME'], name='Random')
color = alt.condition(selection,
                      alt.value('#118C4F'),
                      alt.value('#FFB90D'))

chart1 = alt.Chart(plot_eqr_df).mark_bar().encode(
    x=alt.X('NAME', sort=alt.EncodingSortField('EQR', order='ascending')),
    y='EQR',
    color=color,
    tooltip=['NAME', 'STNAME:N', 'EQR']
).properties(
    title='20 Largest and Smallest Banks by EQR in a Quarter, with Overall Average'
).add_selection(selection)
chart1.encoding.x.title = 'Bank Name (MEAN denotes overall average)'
chart1.encoding.y.title = 'Equity Capital Ratio (EQR)'

In [ ]:
plot_eqr_df_2 = df[df['CERT'].isin(plot_eqr_df['CERT'])]
plot_eqr_df_2 = plot_eqr_df_2.groupby(['CERT', 'QUARTER', 'YEAR']).agg({'QUARTER_YEAR': 'first', 'EQR': 'mean', 'NAME': 'first'}).reset_index()

line = (alt.Chart(plot_eqr_df_2)
         .mark_line(strokeWidth=5, color='#118C4F')
         .encode(x='QUARTER_YEAR:N',
                 y=alt.Y('mean(EQR):Q', scale=alt.Scale(domain=[0,100])),
                 color='NAME:N',
                 tooltip=['NAME', 'QUARTER_YEAR','EQR']
                 )
         ).transform_filter(selection)
line.title='Average EQR by Quarter over Past 30 Years'
line.encoding.x.title='Year and Quarter'
line.encoding.y.title='Equity Capital Ratio (EQR)'
chart1 & line


### Analysis

For this portion of the analysis, I chose to examine banks that represent the top and bottom 20 in average EQR by quarter. For this analysis, I filtered out any banks that do not appear in the data for more than 100 quarters (roughly 80% of the quarters). This is to eliminate any potential outliers in the data. The mean value does not all observations in the data; it only includes all banks with more than 100 quarters of data since Q1 1992. Looking at the histogram, we notice that the average EQR of 11.22 is far closer to the 20 banks that have the lowest equity capital ratio compared to the 20 banks that have the highest equity capital ratio. Looking at the 5 number summary, we see that the average (11.22) is lower than the median (10.27), indicating that there are a few banks with a far higher equity capital ratio than the majority of the rest of the banks in the FDIC database. When we more closely examine the banks that fall in the range of top or bottom 20, we see that there is no consistent state across these banks. It is noteworthy that Michigan has 2 of the 3 highest banks in terms of EQR. 

## Quarters Line Graph

In [ ]:
df_quarters = df.groupby(['QUARTER', 'YEAR']).agg({'QUARTER_YEAR': 'first', 'EQR': 'mean'}).reset_index()

line1 = (alt.Chart(df_quarters)
         .mark_line(strokeWidth=5, color='#118C4F')
         .encode(x='QUARTER_YEAR:N',
                 y=alt.Y('EQR:Q', scale=alt.Scale(domain=[8,14])),
                 tooltip=['QUARTER_YEAR','EQR'])
         )
line1.title='Average EQR by Quarter over Past 30 Years'
line1.encoding.x.title='Year and Quarter'
line1.encoding.y.title='Equity Capital Ratio (EQR)'
line1

## EQR and Assets: Small Counts

In [ ]:
df_eqr_year = df.groupby(['CERT', 'YEAR']).agg({'EQR': 'mean', 'NAME': 'first', 'CITY': 'first', 'STNAME': 'first', 'COUNT': 'sum', 'ASSET': 'mean', 'LIAB': 'mean', 'EQ': 'mean'})

In [ ]:
density_heatmap_df = df_eqr_year[df_eqr_year['ASSET'] <= 500000]

fig = px.density_heatmap(
    density_heatmap_df,
    x="ASSET",
    y="EQR",
    marginal_x="histogram",
    marginal_y="histogram",
    color_continuous_scale=px.colors.sequential.Viridis,
    nbinsx=50,
    nbinsy=1000,
    labels=dict(ASSET="Sum of Assets Owned", EQR="Equity Capital Ratio (EQR)"),
    title="Joint distribution of Average Assets Owned and Average EQR per Year",
    width=650,
    height=650,
    range_x = [1,500000],
    range_y = [0,15]
)

fig.show()

## Bubble Plot

In [ ]:
df_scatter = df.groupby(['CERT','YEAR'])
df_scatter = df.sort_values('NAME')
df_scatter = df_scatter[df_scatter['EQR']>=0]
df_scatter = df_scatter.sort_values('YEAR')

fig = px.scatter(
    df_scatter,
    x='EQ',
    y='EQR',
    animation_frame='YEAR',
    animation_group='NAME',
    size='EQR',
    color='BKCLASS',
    hover_name='NAME',
    template='plotly_white',
    range_color=[-100,100],
    size_max=45
)
fig.update_layout(
    title = 'Bubble Plot of Equity Capital and Equity Capital Ratio by Year',
    xaxis_title = 'Equity Capital (EQ)',
    yaxis_title = 'Equity Capital Ratio (EQR)',
    legend_title_text = 'State Name'
)
fig.show()